In [1]:
import numpy as np
import pandas as pd

In [18]:
df = pd.read_csv("tweets2020.csv",index_col="Unnamed: 0")

In [19]:
df.head()

,2,Classificação
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,0
1,@ShakedownVail Yo. What a #kickass #chill bar ...,0
2,"RT @grazianopascale: ""Parece la trama de un fi...",0
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,2
4,RT @NetoItirussu: Hoje em depoimento da operaç...,0


### Link pra uso do bard:
https://ai.google.dev/gemini-api/docs/get-started/python?hl=pt-br

In [6]:
#pip install -q -U google-generativeai

In [7]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/media/marcos/2caebd18-4175-48de-967d-932bd88604d32/Ajuda Mestrado/tweets/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
#GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [10]:
model = genai.GenerativeModel('gemini-pro')

In [11]:
def classify_tweet(tweet):
    question = "Estou classificando tweets em português brasileiro sobre a bolsa de valores do brasil. Quero que você classifique esse tweet em sentimentos onde cada número é um sentimento: 0- neutro, 1 - positivo, 2- negativo. Esse é o tweet: "+tweet+". Me mande apenas o número da classificação"
    response = model.generate_content(question)
    response = response.text
    for word in response.split():
        if word == '0' or word == '1' or word == '2':
            continue
        else:
            return None
        
    return response


In [12]:
df.head()

,2,Classificação
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,0
1,@ShakedownVail Yo. What a #kickass #chill bar ...,0
2,"RT @grazianopascale: ""Parece la trama de un fi...",0
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,2
4,RT @NetoItirussu: Hoje em depoimento da operaç...,0


1,1 segundo para cada 1 tweet

In [13]:
last_index = 12274
count = 0

indices = df.index[df.index >= last_index]

for index in indices:    
    try:
        if count > 1000:
            break
        else:
            if df.loc[index,'Classificação'] == 10: 
                print(index)  
                tweet = df.loc[index,'2']            
                response = classify_tweet(tweet)
                
                if response == None:
                    continue
            
                else:
                    df.loc[index,'Classificação'] = int(response)
                    count += 1
            
            else: continue
            
    except:
        continue

10000
10001
10002
10003
10004
10005
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10018
10019
10023
10024
10028
10033
10073
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10133
10143
10144
10147
10154
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
10214
10215
10216
10217
10218
10219
10220
10221
10222
10223
10224
10225
10226
10227
10228
10229
10230
10231
10232
10233
10234
10235
10236
10237
10238
10239
10240
10241
10242
10243
10244
10245
10246
10247
10248
10249
10250
10257
10259
10262
10268
10269
10291
10292
10314
10315
10316
10317
10318
10319
10320
10321
10322
10323
10324
10325
10326
10327
10328
10329
10330
10331
10332
10333
10334
10335
10336
1033

In [15]:
df.head(12290)

,2,Classificação
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,0
1,@ShakedownVail Yo. What a #kickass #chill bar ...,0
2,"RT @grazianopascale: ""Parece la trama de un fi...",0
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,2
4,RT @NetoItirussu: Hoje em depoimento da operaç...,0
...,...,...
12285,Para quem acha que o Comunismo e o Nazismo não...,10
12286,"Trabalhe para que isso vá adiante, eu não poss...",10
12287,"por favor , casa comigo",10
12288,RT @DelanJales: Por isso mais uma vez eu digo\...,10


In [23]:
def rate_tweets(df, last_index):
    indices = df.index[df.index >= last_index]

    for index in indices:
        print(df.loc[index,:])
        answer = int(input("Coloque 0 pra continuar, 1 pra alterar classificação e 2 pra parar: "))
        if answer == 0:
            continue
        elif answer == 1:
            right_answer = int(input("Coloque a classicação certa: "))
            df.loc[index,"Classificação"] = right_answer
            print(len(df[df["Classificação"]]))
        else:
            return index

In [24]:
rate_tweets(df,0)

Unnamed: 0                                                       0
2                RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...
Classificação                                                   10
Name: 0, dtype: object


0

In [17]:
df.to_csv("tweets2020.csv")